In [6]:
pip install customtkinter


Note: you may need to restart the kernel to use updated packages.


In [3]:
from customtkinter import CTkImage 
import customtkinter as ctk
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image as PILImage, ImageTk
import numpy as np
import tensorflow as tf
import mysql.connector


model = tf.keras.models.load_model('/Users/owaiskhan/Downloads/Lung Disease Dataset/best_1_lung_disease_model.keras')
CLASS_LABELS = ['Bacterial Pneumonia', 'Corona Virus Disease', 'Normal', 'Tuberculosis', 'Viral Pneumonia']

def resize_image(image_path, max_width, max_height):
    img = PILImage.open(image_path)
    
    # Get original dimensions
    width, height = img.size

    # Calculate the aspect ratio
    aspect_ratio = width / height

    # Determine the new size maintaining the aspect ratio
    if width > height:
        new_width = min(max_width, width)
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = min(max_height, height)
        new_width = int(new_height * aspect_ratio)

    size = (new_width, new_height)
    return size

def get_db_connection():
    return mysql.connector.connect(
        host="localhost", 
        user="root",  
        password="Owais$123",  
        database="lung_disease_detection"
    )

# Preprocess image function
def preprocess_image(image_path):
    try:
        image = PILImage.open(image_path)
    except Exception as e:
        print(f"Error opening image: {e}")
        return None

    image = image.resize((299, 299))
    image = np.array(image) / 255.0
    if len(image.shape) == 2:  
        image = np.stack((image,) * 3, axis=-1)
    image = np.expand_dims(image, axis=0)
    return image

# Function to create a new user profile
def create_profile():
    user_name = name_entry.get().strip()
    if user_name:
        connection = get_db_connection()
        cursor = connection.cursor()
        
        cursor.execute("SELECT * FROM users WHERE user_name = %s", (user_name,))
        result = cursor.fetchone()
        
        if result:
            messagebox.showwarning("Profile Exists", "This username already exists.")
        else:
            cursor.execute("INSERT INTO users (user_name) VALUES (%s)", (user_name,))
            connection.commit()
            messagebox.showinfo("Profile Created", f"Profile for {user_name} created successfully!")
        
        cursor.close()
        connection.close()
    else:
        messagebox.showwarning("Invalid Name", "Please enter a valid username.")

# Function to get user ID from name
def get_user_id(user_name):
    connection = get_db_connection()
    cursor = connection.cursor()
    cursor.execute("SELECT user_id FROM users WHERE user_name = %s", (user_name,))
    result = cursor.fetchone()
    cursor.close()
    connection.close()
    return result[0] if result else None

# Function to save image history to database
def save_history(user_name, image_path, predicted_class):
    user_id = get_user_id(user_name)
    if user_id:
        connection = get_db_connection()
        cursor = connection.cursor()
        cursor.execute(
            "INSERT INTO history (user_id, image_path, predicted_class) VALUES (%s, %s, %s)",
            (user_id, image_path, predicted_class)
        )
        connection.commit()
        cursor.close()
        connection.close()

# Function to view user history
def view_history():
    user_name = name_entry.get().strip()
    user_id = get_user_id(user_name)
    
    if user_id:
        connection = get_db_connection()
        cursor = connection.cursor()
        cursor.execute("SELECT image_path, predicted_class FROM history WHERE user_id = %s", (user_id,))
        history = cursor.fetchall()
        
        if history:
            history_text = f"History for {user_name}:\n\n"
            for i, (file_path, predicted_class) in enumerate(history):
                history_text += f"{i+1}. Image: {file_path} -> Predicted: {predicted_class}\n"
            history_label.configure(text=history_text)
        else:
            messagebox.showinfo("No History", "No history found for this user.")
        
        cursor.close()
        connection.close()
    else:
        messagebox.showinfo("No Profile", "User profile not found. Please create a profile first.")


def predict_class():
    user_name = name_entry.get().strip()
    
    # Check if user has entered a name
    if not user_name:
        messagebox.showwarning("Name Required", "Please enter your name first.")
        return
    
    # Check if the user exists in the database
    user_id = get_user_id(user_name)
    if not user_id:
        messagebox.showwarning("Profile Not Found", "No profile found. Please create a profile first.")
        return

    file_path = filedialog.askopenfilename(initialdir="/Users/owaiskhan/Downloads", title="Select X-Ray Image")
    
    if file_path:
        image = preprocess_image(file_path)
        prediction = model.predict(image)
        class_label = np.argmax(prediction, axis=1)[0]

        # Show the image on the GUI
        img = PILImage.open(file_path)
        img = CTkImage(img, size = resize_image(file_path,200,200))
        image_label.configure(image=img)
        image_label.image = img
        result_label.configure(text=f"Predicted Class: {class_label} , {CLASS_LABELS[class_label]}")

        # Save history if the user has a profile
        save_history(user_name, file_path, CLASS_LABELS[class_label])


# Function for guest mode (no profile)
def guest_mode():
    file_path = filedialog.askopenfilename(initialdir="/Users/owaiskhan/Downloads", title="Select X-Ray Image")

    if file_path:
        image = preprocess_image(file_path)
        prediction = model.predict(image)
        class_label = np.argmax(prediction, axis=1)[0]

        # Show the image on the GUI
        img = PILImage.open(file_path)
        img = CTkImage(img, size = resize_image(file_path,200,200))
        image_label.configure(image=img)
        image_label.image = img
        
        result_label.configure(text=f"Predicted Class: {class_label} , {CLASS_LABELS[class_label]}")


win = ctk.CTk()
win.title("Lung Disease Detection")
win.geometry("800x600")
# win.resizable(False, False)

ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("dark-blue")

# Main frame with padding for content
frame = ctk.CTkScrollableFrame(win, 
                               width=750, height=550, 
                               corner_radius=15, 
                               label_text = "Lung Disease Detection Center",
                               label_fg_color = "blue",
                               # label_font = ("Aptos Narrow",18),
                               # #border_width = 3,
                               # scrollbar_fg_color = "#E56565",
                               # scrollbar_button_color = "",
                               scrollbar_button_hover_color = "#E56565"
                              )
     
frame.pack(pady=20, padx=20)

# Name input field with enhanced styling
name_label = ctk.CTkLabel(frame, text="Enter Your Name (For Profile):", font=("Arial", 14))
name_label.pack(pady=10)

name_entry = ctk.CTkEntry(frame, width=300, font=("Arial", 14))
name_entry.pack(pady=10)

# Create profile button with a better design
create_profile_button = ctk.CTkButton(frame, text="Create Profile", command=create_profile, width=250, height=40, font=("Arial", 14))
create_profile_button.pack(pady=15)

# Buttons organized in a row for better spacing
button_frame = ctk.CTkFrame(frame)
button_frame.pack(pady=20)

select_button = ctk.CTkButton(button_frame, text="Select X-Ray Image (For Registered Users)", command=predict_class, width=200, height=40)
select_button.grid(row=0, column=0, padx=20)

guest_button = ctk.CTkButton(button_frame, text="Select X-Ray Image (Guest Mode)", command=guest_mode, width=200, height=40)
guest_button.grid(row=0, column=1, padx=20)

# Image display area
image_label = ctk.CTkLabel(frame, text="", fg_color = "#564C5E", width=200, height=200)
image_label.pack(pady=10)

# Result display area with larger font
result_label = ctk.CTkLabel(frame, text="Predicted Class: ", font=("Arial", 16), wraplength=500)
result_label.pack(pady=10)

# History section
history_button = ctk.CTkButton(frame, text="View My History", command=view_history, width=200, height=40, font=("Arial", 14))
history_button.pack(pady=20)

history_label = ctk.CTkLabel(frame, text="Your History will be shown here.", font=("Arial", 12), wraplength=500)
history_label.pack(pady=10)

# Start the application
win.mainloop()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception